In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# with open("../data/lidar.pcd", "rb") as f:
#     byte = f.read(1)
#     while byte != b"":
#         # Do stuff with byte.
#         byte = f.read(1)
#         print(byte)



Data from the lidar sensor:
* min_distance: 0.5
* max_distance: 100
* rays: 32
* rotations: 10    
* measurements: 360    
* fov: 41.3300018310547
* angle: 10
* compensated: True 

### Load the point cloud

In [173]:
pcd = o3d.io.read_point_cloud("../data/lidar2.pcd")
print(pcd)
vectorList = np.asarray(pcd.points)
print(vectorList)
# o3d.visualization.draw_geometries([pcd])

PointCloud with 11143 points.
[[-1.97735310e+00  5.76930370e-06 -1.17242849e+00]
 [-1.97728515e+00  3.45212743e-02 -1.17256761e+00]
 [-1.97640276e+00  6.90236911e-02 -1.17257905e+00]
 ...
 [-3.62439728e+01 -3.17092109e+00  6.00578737e+00]
 [-3.59373322e+01 -2.51295590e+00  5.94680309e+00]
 [-3.52411079e+01 -1.23063064e+00  5.82093382e+00]]


In [144]:
pc = o3d.geometry.PointCloud()
# pc.points = o3d.utility.Vector3dVector(arr[:10*360])
pc.points = o3d.utility.Vector3dVector(vectorList[168:193])
print(pc.points)
# test = arr[:360*3]

std::vector<Eigen::Vector3d> with 25 elements.
Use numpy.asarray() to access data.


In [98]:
# test = np.arange(0, 1080)
test = vectorList[0:1080]

 ### Keeps only the points right in front of the vehicle

In [170]:
window = 20
mid = 180
size = 360*20
indexes = np.arange(size)


def keep(index):
    if index % 360 >= mid-window and index % 360 <= mid+window:
        return True
    else:
        return False

# t = np.array([x for (x, i) in zip(arr[:size], indexes) if keep(i)])
infront = np.array([x for (x, i) in zip(vectorList[:size], indexes) if i % 360 >= mid-window and i % 360 <= mid+window])
len(infront)
# test[]
# print(np.vstack([[0,0,0],  infront]))

# NOTE [front|behind, left|right, up|down], right hand system, x, y, z
# infront = np.vstack([[-2,0,-2],[-1,0,-1],[0,0,0],[1,0,1],[2,0,2],  infront]) # Added origo to point cloud
# infront = np.vstack([[0,0,0], [0,0,-2.31],  infront])
infrontCloud = o3d.geometry.PointCloud()
infrontCloud.points = o3d.utility.Vector3dVector(infront)

In [171]:
o3d.visualization.draw_geometries([infrontCloud])

In [167]:
print(np.round(infront[:], 2))

[[ 0.    0.    0.  ]
 [ 2.16  0.   -1.28]
 [ 2.33  0.   -1.31]
 [ 4.33  0.   -2.31]
 [ 4.57  0.   -2.31]
 [ 4.84  0.   -2.31]
 [ 5.14  0.   -2.31]
 [ 5.46  0.   -2.31]
 [ 5.83  0.   -2.31]
 [ 6.23  0.   -2.31]
 [ 6.69  0.   -2.31]
 [ 7.21  0.   -2.31]
 [ 7.82  0.   -2.31]
 [ 8.52  0.   -2.31]
 [ 9.35  0.   -2.31]
 [ 8.1   0.   -1.81]
 [ 8.11  0.   -1.62]
 [ 8.14 -0.14 -1.44]
 [ 8.18 -0.29 -1.25]
 [ 8.89 -0.31 -1.16]
 [ 9.44 -0.49 -1.01]]


In [152]:
frontOfVehicle = np.round(infront[1][0], 2)

for x, y, z in np.round(infront[3:], 2):
    # print(x, y, z)
    if z > -2.3:
        print(x)
        print(x + frontOfVehicle)
        break

8.1
10.43


In [108]:
distances = infrontCloud.compute_mahalanobis_distance() # Mahalanobis distance to each point from center
# approvedPoints = np.array([])
# for point, dis in zip(infront, distances):
#     if (dis > 2 and dis < 10) or dis < 1:
#         if len(approvedPoints) < 1:
#             approvedPoints = np.array([point])
#         else:
#             approvedPoints = np.vstack([approvedPoints, point])
# print(len(approvedPoints))

In [109]:
distances

DoubleVector[3.82202, 1.81903, 1.49692, 1.21212, 1.56673, 1.84884, 1.20578, 1.18331, 1.46058, 1.40839, 1.10234, 1.63076, 0.492814, 0.855643, 0.805753, 0.511464, 1.5531, 1.59855, 1.62028, 2.55861, 2.93765]

### Visualizing

In [157]:
o3d.visualization.draw_geometries([pc])

In [6]:
# Something about normals? 
# See http://www.open3d.org/docs/0.7.0/tutorial/Basic/pointcloud.html#vertex-normal-estimation
pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=10))
o3d.visualization.draw_geometries([pc], point_show_normal=True)

### Only keep points that are within a certain distance
Seems like the distance is not calculated in meters...

In [7]:
mdis = pcd.compute_mahalanobis_distance() # Mahalanobis distance to each point from center
approvedPoints = np.array([])
for point, dis in zip(arr, mdis):
    if (dis > 2 and dis < 10) or dis < 1:
        if len(approvedPoints) < 1:
            approvedPoints = np.array([point])
        else:
            approvedPoints = np.vstack([approvedPoints, point])
print(len(approvedPoints))

9141


In [7]:
np.average(mdis[:360])

0.4576711708063753

In [8]:
# Visualization of new point cloud
newCloud = o3d.geometry.PointCloud()
newCloud.points = o3d.utility.Vector3dVector(approvedPoints)
o3d.visualization.draw_geometries([newCloud])

## Annet

In [3]:
dv = pcd.compute_nearest_neighbor_distance()
np.average(dv)

0.4180552444391524

In [4]:
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=0.01)
o3d.visualization.draw_geometries([downpcd])


Downsample the point cloud with a voxel of 0.05


In [5]:

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd])


Recompute the normal of the downsampled point cloud


compute_mahalanobis_distance

crop(...)
 |      crop(*args, **kwargs)
 |      Overloaded function.
 |      
 |      
 |      1. crop(self, bounding_box)
 |          Function to crop input pointcloud into output pointcloud
 |      
 |      Args:
 |          bounding_box (open3d.geometry.AxisAlignedBoundingBox): AxisAlignedBoundingBox to crop points
 |      
 |      Returns:
 |          open3d.geometry.PointCloud
 |      
 |      2. crop(self, bounding_box)
 |          Function to crop input pointcloud into output pointcloud
 |      
 |      Args:
 |          bounding_box (open3d.geometry.OrientedBoundingBox): AxisAlignedBoundingBox to crop points
 |      
 |      Returns:
 |          open3d.geometry.PointCloud

In [32]:
# help(downpcd)
mdis = pcd.compute_mahalanobis_distance()
print(np.asarray(pcd.points)[0])
print((mdis[0:20]))


[-1.97735310e+00  5.76935463e-06 -1.17242849e+00]
DoubleVector[0.133173, 0.133377, 0.133546, 0.13378, 0.133993, 0.134278, 0.134648, 0.135216, 0.135857, 0.136596, 0.137606, 0.138855, 0.140122, 0.141674, 0.143354, 0.145004, 0.146977, 0.538194, 0.538089, 0.53798]


In [35]:

print("Print a normal vector of the 0th point")
print(downpcd.normals[0])
print("Print the normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])
print("")


Print a normal vector of the 0th point
[0. 0. 1.]
Print the normal vectors of the first 10 points
[[ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 8.22327732e-01  8.39817724e-02 -5.62782518e-01]
 [-1.29269318e-03  4.20271100e-04 -9.99999076e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-7.19252606e-02 -1.67461594e-06 -9.97410024e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-3.36616127e-04 -1.33513029e-02 -9.99910811e-01]
 [-5.83226912e-03  1.02075167e-02 -9.99930893e-01]]



In [47]:
# plt.imshow(np.asarray(downpcd.RGBDImage))
downpcd.compute_mean_and_covariance()

(array([-1.37772025, -0.46522382, -0.80105595]),
 array([[442.5370421 , -20.33351605,  -8.24120481],
        [-20.33351605, 442.45936844,  -3.82447938],
        [ -8.24120481,  -3.82447938,   8.87620334]]))

In [133]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

In [33]:
downpcd[:300]

TypeError: 'open3d.cpu.pybind.geometry.PointCloud' object is not subscriptable